In [337]:
from IPython.display import IFrame
import pylab as plt
%matplotlib inline
import numpy as np
import copy
#%%javascript
#IPython.OutputArea.auto_scroll_threshold = 9999;

# Load data
X_train = np.loadtxt("X_train.csv", delimiter=',', dtype="int", skiprows=1)
y_train = np.loadtxt("y_train.csv", delimiter=',', dtype="int", skiprows=1, usecols=1)
actual = y_train[:,None]
N = X_train.shape[0]

# X_train
# -------
# MYCT,MMIN,MMAX,CACH,CHMIN,CHMAX
# 125,256,6000,256,16,128
# 29,8000,32000,32,8,32
# ...

# Y_train
# -------
# Id,PRP
# 0,198
# 1,269
# ...

def get_weights(X, t):
    XX = np.dot(X.T, X)
    invXX = np.linalg.inv(XX)
    Xt = np.dot(X.T, t)
    return np.dot(invXX, Xt)

def get_mse(actual, prediction, N):
    return np.sqrt(np.sum((actual-prediction)**2)/N)

def loocv(X, T):
    N = X.shape[0]
    loss = 0
    #print(T)
    
    # iterate, leaving one out
    for n in range(0, N):
        X_spliced = np.concatenate((X[:n], X[n+1:]))
        T_spliced = np.concatenate((T[:n], T[n+1:]))
        w = get_weights(X_spliced, T_spliced)
        
        pred = np.dot(X[n], w)
        diff = (pred-T[n])**2
        #print(T[n])
        #print(pred)
        #print(loss)
        loss += diff
    
    return np.sqrt(loss/N)

MYCT = X_train[:,0][:,None]
MMIN = X_train[:,1][:,None]
MMAX = X_train[:,2][:,None]
CACH = X_train[:,3][:,None]
CHMIN = X_train[:,4][:,None]
CHMAX = X_train[:,5][:,None]
PRP = y_train[:,None]

# Model
X = np.hstack((
    np.ones_like(MYCT),
    np.sin(np.log(MYCT)),
    np.sin(np.log(MMIN)),
    np.sin(np.log(MMAX)),
    np.sin(np.log(CACH+0.000001)),
    np.sin(np.log(CHMIN+0.000001)),
    np.sin(np.log(CHMAX+0.000001)),
    np.sin(MYCT**2),
    MMIN**2,
    MMAX**2,
    MYCT,
    MMIN,
    MMAX,
    CACH,
    CHMIN,
    CHMAX    
))
#print(X)

w = get_weights(X, PRP)

prediction = np.dot(X,w)
print("Training loss:", mse(actual, prediction, N))
print("Validation loss:", loocv(X, PRP))






Training loss: 43.09181528690395
Validation loss: [60.28493507]


In [338]:
# Submit, lol
X_test = np.loadtxt("X_test.csv", delimiter=',', dtype="int", skiprows=1)
MYCT = X_test[:,0][:,None]
MMIN = X_test[:,1][:,None]
MMAX = X_test[:,2][:,None]
CACH = X_test[:,3][:,None]
CHMIN = X_test[:,4][:,None]
CHMAX = X_test[:,5][:,None]
N = X_test.shape[0]

# Model
X = np.hstack((
    np.ones_like(MYCT),
    np.sin(np.log(MYCT)),
    np.sin(np.log(MMIN)),
    np.sin(np.log(MMAX)),
    np.sin(np.log(CACH+0.000001)),
    np.sin(np.log(CHMIN+0.000001)),
    np.sin(np.log(CHMAX+0.000001)),
    np.sin(MYCT**2),
    MMIN**2,
    MMAX**2,
    MYCT,
    MMIN,
    MMAX,
    CACH,
    CHMIN,
    CHMAX     
))

print(X[0])
predictions = np.dot(X, w).flatten()
print(predictions.shape)
print(predictions)

pred = np.ones((N, 2))
pred[:,0] = range(N)
pred[:,1] = predictions
np.savetxt('pred.csv', pred, fmt='%d', delimiter=",", header="Id,PRP", comments="")

[ 1.00000000e+00 -8.15013161e-01  9.68146030e-01  4.23749754e-01
 -8.58734369e-01  9.99999500e-07  8.73405021e-01 -2.62083959e-01
  4.00000000e+06  6.40000000e+07  6.00000000e+01  2.00000000e+03
  8.00000000e+03  6.50000000e+01  1.00000000e+00  8.00000000e+00]
(41,)
[100.52708674  26.7871573   86.45661223 297.30184876  47.16384895
  30.82152872  45.84197058 -20.36407962 200.11443933 138.17180576
  33.19778381 547.06718767  45.4423505  168.01656875 361.75711915
  31.35274861  44.86642214  46.91178968  21.07451384  21.82584577
  35.85090947  45.0752696  399.12667027  22.17169235  52.86938988
  62.67276556  42.49608115  42.99151131  33.89689759  34.93333853
  30.72967431  49.89475133  31.21706787  81.88582054 106.36523826
 101.30871505  28.08841862  94.82820004  15.4043776   74.33035584
 198.2667519 ]
